### Import Libraries

In [ ]:
# import required libraries
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import os
import re
from pathlib import Path

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op


### YAML Parameters

In [ ]:
#tag cell with parameters
PROJECT_ID =  ''
DATASET_ID = ''
RESOURCE_BUCKET = ''
FILE_BUCKET = ''
REGION = ''
MODEL_ID = ''
MODEL_NAME = ''
PREDICTION_IMAGE = "northamerica-northeast1-docker.pkg.dev/cio-workbench-image-np-0ddefe/bi-platform/bi-aaaie/images/kfp-pycaret-slim:latest"


In [ ]:
#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
DATASET_ID = 'churn_12_months'
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
MODEL_ID = '5220'
MODEL_NAME = 'churn_12_months'

### Service Parameters

In [ ]:
SERVICE_TYPE = 'churn_12_months'
SERVICE_TYPE_NAME = 'churn-12-months'
TABLE_ID = 'bq_churn_12_months_targets'
REGION = "northamerica-northeast1"

### Pipeline Parameters

In [ ]:
STACK_NAME = 'churn_12_months'
TRAIN_PIPELINE_NAME_PATH = 'churn_12_months_model/training_pipeline'
PREDICT_PIPELINE_NAME_PATH = 'churn_12_months_model/predicting_pipeline'
TRAIN_PIPELINE_NAME = 'churn-12-months-train-pipeline' # Same name as pulumi.yaml
PREDICT_PIPELINE_NAME = 'churn-12-months-predict-pipeline' # Same name as pulumi.yaml
TRAIN_PIPELINE_DESCRIPTION = 'churn-12-months-train-pipeline'
PREDICT_PIPELINE_DESCRIPTION = 'churn-12-months-predict-pipeline'
PIPELINE_ROOT = f"gs://{FILE_BUCKET}"
REGION = "northamerica-northeast1"

In [ ]:
TRAINING_DATASET_TABLE_NAME = 'bq_c12m_training_dataset'
TRAINING_DATASET_SP_NAME = 'bq_sp_c12m_training_dataset'

VALIDATION_DATASET_TABLE_NAME = 'bq_c12m_validation_dataset'
VALIDATION_DATASET_SP_NAME = 'bq_sp_c12m_validation_dataset'

### Save Data Path Parameters

In [ ]:
TRAINING_SAVE_DATA_PATH='gs://{}/{}/{}_train.csv'.format(FILE_BUCKET, SERVICE_TYPE, SERVICE_TYPE)
VALIDATION_SAVE_DATA_PATH='gs://{}/{}/{}_validation.csv'.format(FILE_BUCKET, SERVICE_TYPE, SERVICE_TYPE)
SERVING_SAVE_DATA_PATH='gs://{}/{}/{}_score.csv'.format(FILE_BUCKET, SERVICE_TYPE, SERVICE_TYPE)

### Import Pipeline Components

In [ ]:
# download required component files to local
prefix = f'{STACK_NAME}/{TRAIN_PIPELINE_NAME_PATH}/components/'
dl_dir = 'components/'

storage_client = storage.Client()
bucket = storage_client.bucket(RESOURCE_BUCKET)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs: # download each file that starts with "prefix" into "dl_dir"
    if blob.name.endswith("/"):
        continue
    file_split = blob.name.split(prefix)
    file_path = f"{dl_dir}{file_split[-1]}"
    directory = "/".join(file_path.split("/")[0:-1])
    Path(directory).mkdir(parents=True, exist_ok=True)
    blob.download_to_filename(file_path) 

# import main pipeline components
from components.bq_create_dataset import bq_create_dataset
from components.preprocess import preprocess
from components.train_and_save_model import train_and_save_model
# from components.train_and_save_model_pycaret import train_and_save_model_pycaret
from components.upload_model import upload_model


### Import Pipeline Utils

In [ ]:
# download required component files to local
prefix = f'{STACK_NAME}/{TRAIN_PIPELINE_NAME_PATH}/utils/'
dl_dir = 'utils/'

storage_client = storage.Client()
bucket = storage_client.bucket(RESOURCE_BUCKET)
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs: # download each file that starts with "prefix" into "dl_dir"
    if blob.name.endswith("/"):
        continue
    file_split = blob.name.split(prefix)
    file_path = f"{dl_dir}{file_split[-1]}"
    directory = "/".join(file_path.split("/")[0:-1])
    Path(directory).mkdir(parents=True, exist_ok=True)
    blob.download_to_filename(file_path) 

from utils.monitoring import generate_data_stats
from utils.monitoring import validate_stats 
from utils.monitoring import visualize_stats


### Date Parameters

In [ ]:
scoringDate = date(2021, 9, 1)  # date.today() - relativedelta(days=2)- relativedelta(months=30)
valScoringDate = date(2022, 9, 1)  # scoringDate - relativedelta(days=2)

# training dates
SCORE_DATE = scoringDate.strftime('%Y%m%d')  # date.today().strftime('%Y%m%d')
SCORE_DATE_DASH = scoringDate.strftime('%Y-%m-%d')
SCORE_DATE_MINUS_6_MOS_DASH = ((scoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_START_DASH = (scoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_LAST_MONTH_END_DASH = ((scoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START = (scoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END = (scoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

# validation dates
SCORE_DATE_VAL = valScoringDate.strftime('%Y%m%d')
SCORE_DATE_VAL_DASH = valScoringDate.strftime('%Y-%m-%d')
SCORE_DATE_VAL_MINUS_6_MOS_DASH = ((valScoringDate - relativedelta(months=6)).replace(day=1)).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_START_DASH = (valScoringDate.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m-%d')
SCORE_DATE_VAL_LAST_MONTH_END_DASH = ((valScoringDate.replace(day=1)) - timedelta(days=1)).strftime('%Y-%m-%d')
PROMO_EXPIRY_START_VAL = (valScoringDate.replace(day=1) + relativedelta(months=3)).replace(day=1).strftime('%Y-%m-%d')
PROMO_EXPIRY_END_VAL = (valScoringDate.replace(day=1) + relativedelta(months=4)).replace(day=1).strftime('%Y-%m-%d')

SCORE_DATE_DELTA = 0
SCORE_DATE_VAL_DELTA = 0
TICKET_DATE_WINDOW = 30  # Days of ticket data to be queried


### Model Monitoring Parameters

In [ ]:
MODEL_MONITORING_STACK_NAME = 'util'
MODEL_MONITORING_PATH = 'pipeline_utils'
TRAINING_PIPELINE_NAME_PATH = 'churn_12_months_model/training_pipeline'
SERVING_PIPELINE_NAME_PATH = 'churn_12_months_model/serving_pipeline'

In [ ]:
today = date.today()

# BQ table where training data is stored
INPUT_TRAINING_DATA_TABLE_PATH = f"{PROJECT_ID}.{DATASET_ID}.{TRAINING_DATASET_TABLE_NAME}"
INPUT_TRAINING_DATA_CSV_PATH = 'gs://{}/{}/{}_train.csv'.format(FILE_BUCKET, SERVICE_TYPE, SERVICE_TYPE)                               

# BQ dataset where monitoring stats are stored
MODEL_MONITORING_DATASET = "churn_12_months"

# Paths to statistics artifacts in GCS
TRAINING_STATISTICS_PATH = f"gs://{FILE_BUCKET}/{STACK_NAME}/{TRAINING_PIPELINE_NAME_PATH}/training_statistics/training_statistics_{today}"
TRAINING_STATS_PREFIX = f"{STACK_NAME}/statistics/training_statistics"
TRAINING_STATISTICS_OUTPUT_PATH = f"gs://{FILE_BUCKET}/{STACK_NAME}/statistics/training_statistics_{today}" 

ANOMALIES_PATH = f"gs://{FILE_BUCKET}/{STACK_NAME}/anomalies/anomalies_{today}"
PREDICTION_ANOMALIES_PATH = f"gs://{FILE_BUCKET}/{STACK_NAME}/anomalies/prediction_anomalies_{today}"
PREDICTION_STATS_PATH = f"gs://{FILE_BUCKET}/{STACK_NAME}/statistics/prediction_statistics_{today}"
PREDICTION_STATS_PREFIX = f"{FILE_BUCKET}/statistics/prediction_statistics"

# Paths to schemas in GCS
SCHEMA_PATH = f'gs://{FILE_BUCKET}/{STACK_NAME}/schemas/training_stats_schema_{today}'
# SATISTICS_PATH = f'gs://{FILE_BUCKET}/{STACK_NAME}/schemas/training_statistics_{today}'
# Thresholds for anomalies
ANOMALY_THRESHOLDS_PATH = f"{STACK_NAME}/{TRAINING_PIPELINE_NAME_PATH}/training_statistics/anomaly_thresholds.json" #same path structure as utils reading from bucket

# Filters for predictions monitoring
DATE_COL = 'partition_date'
DATE_FILTER = str(today)
TABLE_BLOCK_SAMPLE = 1 # no sampling
ROW_SAMPLE = 1 # no sampling

### Pipeline

In [ ]:
# library imports
from kfp.v2 import compiler
from google.cloud.aiplatform import pipeline_jobs
@dsl.pipeline(
    name=TRAIN_PIPELINE_NAME, 
    description=TRAIN_PIPELINE_DESCRIPTION
    )
def pipeline(
        project_id: str = PROJECT_ID,
        region: str = REGION,
        resource_bucket: str = RESOURCE_BUCKET, 
        file_bucket: str = FILE_BUCKET
    ):
    
    #### this code block is only for a personal workbench 
    
    import google.oauth2.credentials
    token = !gcloud auth print-access-token
    token_str = token[0]
    
    #### the end
    
    from datetime import datetime
    update_ts = datetime.now()
    update_ts_str = update_ts.strftime('%Y-%m-%d %H:%M:%S')
    
    # ----- create training set --------
    bq_create_training_dataset_op = bq_create_dataset(score_date=SCORE_DATE_DASH
                                  , score_date_delta=SCORE_DATE_DELTA
                                  , project_id=PROJECT_ID
                                  , dataset_id=DATASET_ID
                                  , region=REGION
                                  , environment='training'
                                  , token=token_str
                                  )

    bq_create_training_dataset_op.set_memory_limit('32G')
    bq_create_training_dataset_op.set_cpu_limit('4')
    
    # ----- preprocessing train data --------
    preprocess_train_op = preprocess(pipeline_dataset=TRAINING_DATASET_TABLE_NAME
                                    , save_data_path=TRAINING_SAVE_DATA_PATH
                                    , project_id=PROJECT_ID
                                    , dataset_id=DATASET_ID
                                    , score_date_dash=SCORE_DATE_DASH
                                    )

    preprocess_train_op.set_memory_limit('64G')
    preprocess_train_op.set_cpu_limit('8')
    
    # ----- create validation set --------
    bq_create_validation_dataset_op = bq_create_dataset(score_date=SCORE_DATE_VAL_DASH
                                      , score_date_delta=SCORE_DATE_VAL_DELTA
                                      , project_id=PROJECT_ID
                                      , dataset_id=DATASET_ID
                                      , region=REGION
                                      , environment='validation'
                                      , token=token_str
                                      )

    bq_create_validation_dataset_op.set_memory_limit('32G')
    bq_create_validation_dataset_op.set_cpu_limit('4')
    
    # ----- preprocessing validation data --------
    preprocess_validation_op = preprocess(pipeline_dataset=VALIDATION_DATASET_TABLE_NAME
                                        , save_data_path=VALIDATION_SAVE_DATA_PATH
                                        , project_id=PROJECT_ID
                                        , dataset_id=DATASET_ID
                                        , score_date_dash=SCORE_DATE_VAL_DASH
                                        )

    preprocess_validation_op.set_memory_limit('64G')
    preprocess_validation_op.set_cpu_limit('8')

    train_and_save_model_op = train_and_save_model(file_bucket=FILE_BUCKET
                                                   , service_type=SERVICE_TYPE
                                                   , project_id=PROJECT_ID
                                                   , dataset_id=DATASET_ID
                                                   , token=token_str
                                                   )

    train_and_save_model_op.set_memory_limit('64G')
    train_and_save_model_op.set_cpu_limit('16')
    
    # col_input_op = col_list = bq_create_dataset_op.outputs["col_list"]
    upload_model_op = upload_model(project_id = PROJECT_ID
                                , region = REGION
                                , model = train_and_save_model_op.outputs["model"]
                                , model_name = MODEL_NAME
                                , prediction_image = PREDICTION_IMAGE
                                , col_list = train_and_save_model_op.outputs["col_list"]
                                , model_uri = train_and_save_model_op.outputs["model_uri"]
                                )
    
    upload_model_op.set_memory_limit('32G')
    upload_model_op.set_cpu_limit('4')
        
    generate_training_data_stats_op = generate_data_stats(project_id=PROJECT_ID
                                                        , bucket_nm=FILE_BUCKET
                                                        , data_type = 'csv'
                                                        , op_type = 'training'
                                                        , model_nm = MODEL_NAME
                                                        , update_ts = update_ts_str
                                                        , token = token_str
                                                        , model_type  = 'supervised'
                                                        , dest_stats_gcs_path = TRAINING_STATISTICS_OUTPUT_PATH
                                                        , src_csv_path = INPUT_TRAINING_DATA_CSV_PATH
                                                        , in_bq_ind = True
                                                        , dest_schema_path = SCHEMA_PATH
                                                        , dest_stats_bq_dataset = MODEL_MONITORING_DATASET
                                                        , pass_through_features = ['ban']
                                                        , training_target_col = 'target'
                                                        ).set_display_name("generate-training-data-statistics")

    generate_training_data_stats_op.set_memory_limit('32G')
    generate_training_data_stats_op.set_cpu_limit('4')
    
    # visualize serving statistics
    visualize_training_stats_op = visualize_stats(statistics = generate_training_data_stats_op.outputs["statistics"] 
                                                , stats_nm=f"Training Data Statistics {update_ts_str}"
                                                , op_type = 'training'
                                                ).set_display_name("visualize-Training-data-statistics")
        
    preprocess_train_op.after(bq_create_training_dataset_op)
    bq_create_validation_dataset_op.after(preprocess_train_op)
    preprocess_validation_op.after(bq_create_validation_dataset_op)
    
    train_and_save_model_op.after(preprocess_validation_op)
    upload_model_op.after(train_and_save_model_op)
    
    generate_training_data_stats_op.after(preprocess_validation_op)
    visualize_training_stats_op.after(generate_training_data_stats_op)
    

### Run the Pipeline Job

In [ ]:
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs
# import json

# compiler.Compiler().compile(
#    pipeline_func=pipeline, package_path="pipeline.json"
# )

# job = pipeline_jobs.PipelineJob(
#                                    display_name=TRAIN_PIPELINE_NAME,
#                                    template_path="pipeline.json",
#                                    location=REGION,
#                                    enable_caching=False,
#                                    pipeline_root = PIPELINE_ROOT
#                                 )
# job.run(service_account = f"bilayer-sa@{PROJECT_ID}.iam.gserviceaccount.com")


In [ ]:
import google.oauth2.credentials
import json

token = !gcloud auth print-access-token
CREDENTIALS = google.oauth2.credentials.Credentials(token[0])

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path="pipeline.json"
)

job = pipeline_jobs.PipelineJob(
   display_name=TRAIN_PIPELINE_NAME,
   template_path="pipeline.json",
   credentials = CREDENTIALS,
   pipeline_root = PIPELINE_ROOT,
   location=REGION,
   enable_caching=False # I encourage you to enable caching when testing as it will reduce resource use
)

job.run()